# Boosting

É uma das ideias de aprendizagem mais poderosas introduzidas nos ultimos vinte anos. Ela foi originalmente projetada para problemas de classificaçao, mas pode ser estendida para regressao. A motivação por tras do boosting foi ter umprocedimento que combina as saidas de muitos classificadores "fracos" para produzir um "comite" poderoso. A partir desta perspectiva, o boosting tem uma semelhança com o bagging. No entanto, podemos notar uma conexao superficial e que o boosting é fundamentalmente diferente.  

## AdaBosst:

Cinco grandes diferenças entre AdaBosst e Random Forest são:  
I. Enquanto no RF há uma floresta de arvores de tamanhos e proporções diferentes, no AdaBost há uma foresta de "stamps", arvores de 1 de profundidade e 2 folhas.  
II. Enquanto no RF as arvores são criadas paralelamente, sem que uma interfira no resultado de outra, no AdaBoost uma arvore complementa o resultado da outr.  
III. Equanto no RF as respostas de cada arvore tem o "mesmo" peso e a resposta final é "escolhida" de forma "democratica", no AdaBoost cada resposta tem seu peso.  
IV. Equanto no RF é feito uma comparação de frequencia ou uma média dependendo se for qualitativo ou quantitativo, no AdaBoost é feito a soma dos "pesos" de cada classificador, assim a maior soma se torna o resultado.  
V. O AdaBoost é mais sensivel a dados ruidosos ou outliers, ja que tende a sempre tentar corrigir os dados com mais erros, ja o RF é mais robusto quanto a isso, ja que a média das previsoes de varias arvores tende a diminuir outliers.


In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [58]:
titanic = sns.load_dataset('titanic')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [60]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [62]:
titanic = titanic.drop(columns=['alive', 'deck'])
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  embark_town  889 non-null    object  
 12  alone        891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.5+ KB


In [64]:
titanic = titanic.dropna()
titanic = pd.get_dummies(titanic, drop_first=True).astype(int)
titanic.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   survived                 712 non-null    int32
 1   pclass                   712 non-null    int32
 2   age                      712 non-null    int32
 3   sibsp                    712 non-null    int32
 4   parch                    712 non-null    int32
 5   fare                     712 non-null    int32
 6   adult_male               712 non-null    int32
 7   alone                    712 non-null    int32
 8   sex_male                 712 non-null    int32
 9   embarked_Q               712 non-null    int32
 10  embarked_S               712 non-null    int32
 11  class_Second             712 non-null    int32
 12  class_Third              712 non-null    int32
 13  who_man                  712 non-null    int32
 14  who_woman                712 non-null    int32
 15  embark_town

In [66]:
titanic.head()

,survived,pclass,age,sibsp,parch,fare,adult_male,alone,sex_male,embarked_Q,embarked_S,class_Second,class_Third,who_man,who_woman,embark_town_Queenstown,embark_town_Southampton
0,0,3,22,1,0,7,1,0,1,0,1,0,1,1,0,0,1
1,1,1,38,1,0,71,0,0,0,0,0,0,0,0,1,0,0
2,1,3,26,0,0,7,0,1,0,0,1,0,1,0,1,0,1
3,1,1,35,1,0,53,0,0,0,0,1,0,0,0,1,0,1
4,0,3,35,0,0,8,1,1,1,0,1,0,1,1,0,0,1


In [68]:
y = titanic.survived
X = titanic.drop(columns='survived')
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)

In [70]:
base_classifier = DecisionTreeClassifier(max_depth=1)
ada = AdaBoostClassifier(estimator=base_classifier, n_estimators=100, algorithm='SAMME', random_state=45)

ada.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME',
                   estimator=DecisionTreeClassifier(max_depth=1),
                   n_estimators=100, random_state=45)

In [72]:
y_pred = ada.predict(X_test)
accuracy_score(y_test, y_pred)

0.7921348314606742

In [74]:
scores = cross_val_score(ada, X, y, cv=5)
scores.mean()

0.8089825667290456

# 5 hyperparametros:

n_estimators: número de aprendizes fracos.  
learning_rate: controla a contribuição dos aprendizes fracos.  
algorithm: tipo de algoritmo de bossting a ser utilizado.  
random_state : semente para gerar numeros aleatorios.  
estimator: estimador base a partir do qual o ensemble aumentado é construído.